# Lesson 6 — MCMC Convergence Diagnostics

A practical tour of tools to check whether your Markov chain has reached its stationary (posterior) distribution.


1. **Trace plots** — detect trends and wandering behaviour  
2. **Autocorrelation & ESS** — quantify dependency and effective information size  
3. **Burn‑in** — decide how many early draws to discard  
4. **Multiple chains & $\hat R$** — Gelman–Rubin shrink factor for convergence assurance  
5. Quick posterior estimates once diagnostics look satisfactory

## 1  Setup

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import arviz as az
from statsmodels.tsa.stattools import acf


In [ ]:

# Helper MH sampler
def mh(n, ybar, n_iter=1000, mu_init=0.0, cand_sd=0.9):
    mu = np.empty(n_iter)
    mu_now = mu_init
    lg_now = -0.5*n*(mu_now-ybar)**2
    accpt = 0
    for i in range(n_iter):
        mu_cand = np.random.normal(mu_now, cand_sd)
        lg_cand = -0.5*n*(mu_cand-ybar)**2
        if np.random.rand() < np.exp(lg_cand-lg_now):
            mu_now, lg_now = mu_cand, lg_cand
            accpt += 1
        mu[i] = mu_now
    return {'mu': mu, 'accpt': accpt/n_iter}


## 2  Trace plots

In [ ]:

np.random.seed(61)
n, ybar = 30, 1.0
post0 = mh(n, ybar, n_iter=10000, cand_sd=0.9)
print('acceptance =', post0['accpt'])

plt.figure(figsize=(6,2.5))
plt.plot(post0['mu'][500:], lw=0.6)
plt.xlabel('Iteration'); plt.ylabel('$\mu$')
plt.title('Trace plot'); plt.show()


## 3  Effective sample size

In [ ]:

ess = az.ess(post0['mu'])
print('ESS ≈', ess.round(0))
az.plot_autocorr(post0['mu'], max_lag=50)


## 4  Burn‑in

In [ ]:

np.random.seed(62)
post_burn = mh(n, ybar, n_iter=500, mu_init=10.0, cand_sd=0.3)
plt.figure(figsize=(6,2.5))
plt.plot(post_burn['mu'], lw=0.6)
plt.axvline(100, color='red', ls='--')
plt.xlabel('Iteration'); plt.ylabel('$\mu$'); plt.title('Burn‑in example')
plt.show()


## 5  Multiple chains · $\hat R$

In [ ]:

starts = [15.0, -5.0, 7.0, 23.0, -17.0]
chains = [mh(n, ybar, n_iter=500, mu_init=s, cand_sd=0.4)['mu'] for s in starts]
idata = az.convert_to_inference_data(np.array(chains))
print('R‑hat =', float(az.rhat(idata).sel(chain_draw='mu')))
az.plot_trace(idata, compact=True, figsize=(8,4))


## 6  Posterior estimation

In [ ]:

mu_keep = post0['mu'][1000:]
print('Pr(μ > 1) ≈', np.mean(mu_keep > 1).round(3))
